In [2]:
# REF: https://arxiv.org/pdf/0704.1317.pdf

# import libraries
import numpy as np
from scipy import interpolate
import scipy

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:

# Create an integer messege vector b

b = np.array([3,6,7,2,5,2])

# create H here n = 6 d = 3 normalize H to get |det(G)| = |det(H)| = 1

H = np.array([[0, -0.8, 0, -0.5, 1, 0],[0.8, 0, 0, 1, 0, -0.5],[0, 0.5, 1, 0, 0.8, 0],[0, 0, -0.5, -0.8, 0, 1],[1, 0, 0, 0, 0.5, 0.8],[0.5, -1, -0.8, 0, 0, 0]])

H_norm = H/np.abs(np.linalg.det(H))**(1/6.)

# Calculate generator matrix G = inv(H) normalize H to get |det(G)| = 1

G = np.linalg.inv(H_norm)

# Calculate codeword x

x = np.dot(G,b)

x = x.reshape(-1,1)

# create noisy codeword y = x + w

mu, sigma = 0, 10 # mean and standard deviation
w = np.random.normal(mu, sigma, x.shape)
y = x + w

# y is user input fingerprint
x_input = np.arange(-200.0,200.1,0.1)

In [16]:
# x1, x2, x3, x4,...,xn -> variable node c1,c2,....,cn -> check node
# Initialization

def message(x,k):
    # y and sigma should be given
    # produces f_k^0(x)
    """
    initial message vector
    
    INPUT
    x (1d array) - input suport vactor
    k (int) - kth node of x
    
    OUTPUT
    gaussian like array
    """
    global y
    global sigma
    return (np.exp(-(((y[k] - x)/sigma)**2)/2.))/(np.sqrt(2*np.pi*sigma**2))


def extrapolate(y,x_in):
    """
    Given an array, returns the extrapolated function f(x_in)
    y = f(arr)
    
    INPUT
    x_in (1d array) - range of the array
    y (1d array) - array to be interpolated
    
    OUTPUT
    interpolated funtion f(arr)
    """
    f = interpolate.interp1d(arr, y, fill_value = "extrapolate")
    return f

#Check node will receive a list of varibale node messages.

class CheckNode:
    def __init__(self,h,j):
        """
        initialize CheckNode class

        INPUT
        h (array) - h is the elements of a row from H ex: [0,-0.8,0,-0.5,1,0]
        j (int) - jth check node
        """
        self.h = h
        self.j = j
        
    def conv(self,arr,x):
        """
        convolve all messages except f_j(x)
        arr(1d array) - received message vector from variable node
        x (array) - input suport vactor
        
        Returns p_j(x)
        """
        #interpolate "arr"
        mess = extrapolate(arr,x)
        p_j = 1
        for i,p in enumerate (self.h[:self.j]):
            if p != 0:
                p_j = np.convolve(p_j,mess(x/p,i),mode='full')
        for l,m in enumerate (self.h[self.j+1:]):
            if m != 0:
                p_j = np.convolve(p_j,mess(x/m,i),mode='full')
        return p_j

    def stretch(self,x):
        """
        The result is stretched by -h_j
        x (array) - input suport vactor
        
        RETURNS p_j(-h_j*x)
        """
        return CheckNode.conv(self.j, self.h,-self.h[self.j]*x)
    
    def periodic_extension(self,x):
        """
        The result is extended to a periodic function with period 1/|hj|:
        x (array) - input suport vactor
        
        RETURNS Q_j(x)
        """
        h = np.abs(self.h)
        minval = h[np.nonzero(h)].min()
        end = np.rint(x.max()*minval) # 100
        i = np.rint(x.min()*minval) # -100
        q = 1
        while i < end:
            q += CheckNode.stretch(self, x - i/self.h[self.j])
            i += 1
        return q
    def Q(self,x):
        """        
        RETURNS Q_j(x)
        """
        return CheckNode.periodic_extension(self,x)
    
    

class VariableNode(CheckNode):
    def __init__(self,k):
        pass
    def product(self):
        pass
    def normalization(self,):
        pass
    def f(self):
        pass

In [17]:
node = CheckNode([0, -0.8, 0, -0.5, 1, 0],1)

In [4]:
def _bitsandnodes(H):
    """Return bits and nodes of a parity-check matrix H."""
    if type(H) != scipy.sparse.csr_matrix:
        bits_indices, bits = np.where(H)
        nodes_indices, nodes = np.where(H.T)
    else:
        bits_indices, bits = scipy.sparse.find(H)[:2]
        nodes_indices, nodes = scipy.sparse.find(H.T)[:2]
    bits_histogram = np.bincount(bits_indices)
    nodes_histogram = np.bincount(nodes_indices)

    return bits_histogram, bits, nodes_histogram, nodes


In [5]:
_,_,nh,n = _bitsandnodes(H)

In [15]:
nodes_indices, nodes = np.where(H.T)

In [33]:
scipy.sparse.find(H.T)

(array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5], dtype=int32),
 array([1, 4, 5, 0, 2, 5, 2, 3, 5, 0, 1, 3, 0, 2, 4, 1, 3, 4], dtype=int32),
 array([ 0.8,  1. ,  0.5, -0.8,  0.5, -1. ,  1. , -0.5, -0.8, -0.5,  1. ,
        -0.8,  1. ,  0.8,  0.5, -0.5,  1. ,  0.8]))

In [39]:
np.where(H.T)[0].reshape(6, -1)

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3],
       [4, 4, 4],
       [5, 5, 5]])

In [40]:
np.zeros(shape=(6, 6, 3))

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [16]:
nodes_indices

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5])